# Usage

## Categorizations

### Included categorizations

In the `climate_categories` package, the categorizations are available
directly at the top-level namespace, and as a dictionary in `.cats`:

In [ ]:
import climate_categories

climate_categories.cats

In [ ]:
climate_categories.IPCC2006

In [ ]:
climate_categories.cats["IPCC2006"]

Metadata for each categorization are accessible as properties:

In [ ]:
print(climate_categories.IPCC2006.name)
print(climate_categories.IPCC2006.title)
print(climate_categories.IPCC2006.comment)
print(climate_categories.IPCC2006.references)
print(climate_categories.IPCC2006.institution)
print(climate_categories.IPCC2006.last_update)
print(climate_categories.IPCC2006.version)

The categorization can be used as a dictionary mapping category codes to categories:

In [ ]:
climate_categories.IPCC2006["1.A"]

You can also query using alternative spellings of the code:

In [ ]:
climate_categories.IPCC2006["1A"]

For the categories, metadata is also available: a `title`, maybe a
`comment`, all of its `codes` and possibly additional non-standard information
in the `info` dictionary:

In [ ]:
one_a = climate_categories.IPCC2006["1.A"]
print(one_a.title)
print(one_a.comment)
print(one_a.codes)
print(one_a.info)

For hierarchical categorizations, you can also query for parent and child
categories.
Note that a list of sets of children is returned in case a category
can be composed differently:

In [ ]:
climate_categories.IPCC2006["1.A"].children

In [ ]:
climate_categories.IPCC2006["1.A"].parents

Finally, you can check if a categorization is hierarchical, and
for hierarchical categorizations you can check if the sum of all
child categories should be equal to the sum of parent categories:

In [ ]:
print(f"Hierachical: {climate_categories.IPCC2006.hierarchical}")
print(f"Total sum: {climate_categories.IPCC2006.total_sum}")

### Visualization

The relationships between categories in a hierarchical categorization can be visualized
in a tree-like fashion:

In [ ]:
# Limit the maximum depth shown using `maxdepth`
print(climate_categories.IPCC2006.show_as_tree(maxdepth=2))

In [ ]:
# Print only a part of tree using `root`
print(climate_categories.IPCC2006.show_as_tree(root="1A1"))

### Child sets in hierarchical categorizations

For hierarchical categorizations, it is possible that a category can be composed of multiple child sets.
As an example, in emissions reporting it is possible to report industrial emissions either by industry sectors, or by fuel.
In this case, the parent `industry` category has two sets of children: either all the industry sectors, or all of the fuels.

You can see this with two toy example categorizations included in `climate_categories`:

In [ ]:
import climate_categories.tests.examples

HierEx = climate_categories.tests.examples.HierEx()
print(
    "Hierarchical categorization with only one way to subdivide the top category:"
)
print(HierEx.show_as_tree())

HierAltEx = climate_categories.tests.examples.HierAltEx()
print(
    "\nHierarchical categorization with two ways to subdivide the top category:"
)
print(HierAltEx.show_as_tree())

As you can see, alternative ways to subdivide a category are indicated with double lines in `show_as_tree`.
Programmatically, the difference is clear because `children` contains a list of possible child sets:

In [ ]:
HierEx["0"].children

In [ ]:
HierAltEx["0"].children

### Finding leaf descendants

For purposes like re-calculating top-level categories from simple leaf categories, it
is useful to find the descendants of a category which have no children. Use the
`leaf_children` property to do so.

In [ ]:
HierEx["0"].leaf_children

### Extending categorizations

Often, you want to use a common categorization, but for one reason or
another, you have to add a couple of categories. This is possible:

In [ ]:
IPCC2006_lulucf_extra = climate_categories.IPCC2006.extend(
    name="IPCC2006_lulucf_extra",
    categories={
        "M0.EL": {
            "title": "Total excluding lulucf",
            "comment": "All emissions and removals except emissions from land use, land use change, and forestry",
        }
    },
    children=[("M0.EL", ("1", "2", "4", "5"))],
)

print(IPCC2006_lulucf_extra.name)
print(IPCC2006_lulucf_extra.title)
print(IPCC2006_lulucf_extra.comment)
print(IPCC2006_lulucf_extra.show_as_tree(maxdepth=2))

If the canonical top level category of hierarchical
categorizations is defined, you can also calculate the level of a category in the
hierarchy:

In [ ]:
print(climate_categories.IPCC2006["0"].level)
print(climate_categories.IPCC2006["1.A"].level)

### Pandas integration

For each categorization, the categories are also available as a pandas
DataFrame:

In [ ]:
climate_categories.IPCC2006.df

### Finding unknown codes

Searching for a code in all included categorizations is possible
using the `find_code` function:

In [ ]:
climate_categories.find_code("1A")

## Conversions

### Included conversions

You can get the rules for conversion between categories using
the conversion objects:

In [ ]:
conv = climate_categories.IPCC1996.conversion_to("IPCC2006")
conv

The conversion object can be queried for metadata:

In [ ]:
print(conv.categorization_a)
print(conv.categorization_b)
print(conv.auxiliary_categorizations)
print(conv.comment)
print(conv.institution)
print(conv.references)
print(conv.version)
print(conv.last_update)

More importantly, the conversion object also holds the actual conversion rules:

In [ ]:
len(conv.rules)

In [ ]:
conv.rules[0]

For the rules, the most important parts are the categories and factors for each side and the specification for which auxiliary categories the rule is valid:

In [ ]:
rule = conv.rules[0]
print(rule.factors_categories_a)
print(rule.factors_categories_b)
print(rule.auxiliary_categories)

In this example, the category 1 of the IPCC1996 categorization equals exactly the category 1 of the IPCC2006 categorization (i.e. for both, the factor is unity) and the rule is valid for all gases (the set of gases is empty, meaning an unrestricted rule).

Often, you might be interested in rules concerning a specific set of categories, which you can also fetch:

In [ ]:
one_or_two_rules = conv.relevant_rules(
    {climate_categories.IPCC1996["1"], climate_categories.IPCC1996["2"]}
)

for rule in one_or_two_rules:
    print("###")
    print(rule.format_human_readable())

Here, we used the `format_human_readable()` function to get a nicely formatted output.
You can see that for this conversion, the category 1 maps cleanly between the two categorizations, but for category 2, you need to combine categories 2 and 3 from IPCC1996 to get category 2 from IPCC2006.

### Finding inconsistencies

Dealing with large conversion rule sets can be daunting, so there are a few functions to help when checking conversions:

In [ ]:
# describe all of the conversion rules
# We only print the start of the description because it is looong
print(conv.describe_detailed()[:200] + "…")

In [ ]:
# Find potential over counting problems
suspected_problems = conv.find_over_counting_problems()
for p in suspected_problems:
    print(p)
    print()

Note that `find_over_counting_problems` at the moment can't reliably detect all over counting problems and also some suspected problems might be fine under closer examination.
Use this function only to generate hints for possible problems.

In [ ]:
# Find unmapped categories
missing_1996, missing_2006 = conv.find_unmapped_categories()
# returns sets of missing categories
# unfortunately, this conversion is not very complete:
print(len(missing_1996))
print(len(missing_2006))

In [ ]:
# the sets just contain regular categories which were forgotten
next(iter(missing_1996))

### Viewing the full conversion

To view the full conversion, also consider directly loading the source CSVs from the `climate_categories/data/` folder in a spreadsheet program.